# Modelos Temperaturas

# Apuntes
- un modelo que pueda predecir las 6 variables al mismo tiempo o 3 para los 3 tipos de temperatura ?
- podría usar uno de deep learning con pytorch, debería ser una RNN dado que son en series temporales, lo haré sobre los días o sobre la semana estadística ?
    - usaremos una GRU

Podría crear un webscrapper para sacar la data de esta página https://climatologia.meteochile.gob.cl/application/requerimiento/producto/RE2006, pero para demostrar habilidades primero probaré con un modelo de deep learning

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset
import torch.optim as optim

device = 'cuda'     # o puede ser "cpu"

Cargar data y crear secuencia

In [ ]:
# Functions 
def create_sequences(df, seq_length):
    xs, ys = [], []
    for i in range(len(df) - seq_length):
        x = df.iloc[i:(i+seq_length), 1]
        y = df.iloc[i+seq_length, 1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Create training datasets
X_train, y_train = create_sequences(train_data, seq_length)
print(X_train.shape, y_train.shape)

# Convert to Torch Dataset
dataset_train = TensorDataset(
    torch.from_numpy(X_train).float(),
    torch.from_numpy(y_train).float()
    )

# Modelo

In [ ]:

class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(GRU, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# Defining hiperparameters
input_size = 10  # Número de características de entrada
hidden_size = 64
output_size = 6  # Número de variables a predecir
num_layers = 2

# Instantiating model
model = GRU(input_size, hidden_size, output_size, num_layers)

# Verifing model
print(model)


Entrenamiento del modelo

In [ ]:
# Definir una función de pérdida
criterion = nn.MSELoss()  # O cualquier otra adecuada para tu problema

# Definir un optimizador
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    
    # Suponiendo que tienes un DataLoader `train_loader`
    for data, targets in train_loader:
        # Mover datos a la GPU si está disponible
        data, targets = data.to(device), targets.to(device)
        
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, targets)
        
        # Backward pass y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')